In [ ]:
from pathlib import Path

from sed import SedProcessor
import sed
import numpy as np

%matplotlib inline
# %matplotlib ipympl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget

In [ ]:
local_path = Path(sed.__file__).parent.parent/'tutorial/'
config_file = local_path/'hextof_config.yaml'
assert config_file.exists()

# Loading Data

In [ ]:
config={"core": {"paths": {
    "data_raw_dir": "/asap3/flash/gpfs/pg2/2023/data/11019101/raw/hdf/offline/fl1user3", 
    "data_parquet_dir": "/home/agustsss/temp/sed_parquet/"
}}}
sp = SedProcessor(runs=[44797], config=config, user_config=config_file, system_config={}, collect_metadata=False)

In [ ]:
sp.add_jitter()
sp.align_dld_sectors()

# time-of-flight spectrum

In [ ]:
sp.append_tof_ns_axis()

In [ ]:
axes = ['sampleBias','dldTime']
bins = [5, 250]
ranges = [[28,33],  [650,800]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
res.plot.line(x='dldTime')

# Energy Calibration

## using lmfit

In [ ]:
axes = ['sampleBias', 'dldTimeSteps']
bins = [5, 500]
ranges = [[28,33], [4000, 4800]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
sp.load_bias_series(binned_data=res)

In [ ]:
ranges=(4120, 4200)
ref_id=0
sp.find_bias_peaks(ranges=ranges, ref_id=ref_id)

In [ ]:
sp.calibrate_energy_axis(
    ref_id=2,
    ref_energy=-1,
    method="lmfit",
    energy_scale='kinetic',
    d={'value':1.,'min': .2, 'max':1.0, 'vary':True},
    t0={'value':5e-7, 'min': 1e-7, 'max': 1e-6, 'vary':True},
    E0={'value': 0., 'min': -100, 'max': 100, 'vary': True},
)

In [ ]:
sp.append_energy_axis()

In [ ]:
sp.dataframe[['dldTime','dldTimeSteps','energy','dldSectorID']].head()

In [ ]:
axes = ['sampleBias', 'energy']
bins = [5, 500]
ranges = [[28,33], [-10,10]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
res.mean('sampleBias').plot.line(x='energy',linewidth=3)
res.plot.line(x='energy',linewidth=1,alpha=.5);

In [ ]:
sp.dataframe

In [ ]:
sp.add_energy_offset(
    constant=0,
    columns=['sampleBias','monochromatorPhotonEnergy','tofVoltage'],
    signs=[1,-1,-1],
    preserve_mean=True,
)

In [ ]:
sp.dataframe

In [ ]:
axes = ['sampleBias', 'energy']
bins = [5, 500]
ranges = [[28,33], [-10,2]]
res_fit = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
ax = plt.subplot(111)
res_fit.energy.attrs['unit'] = 'eV'
res_fit.mean('sampleBias').plot.line(x='energy',linewidth=3, ax=ax)
res_fit.plot.line(x='energy',linewidth=1,alpha=.5,label='all',ax=ax);

### save the calibration parameters
This is currently overwriting all other values in the config file you point at. Do not overwrite local settings! 
This should be fixed in the future.

In [ ]:
# sp.save_energy_calibration(filename=local_path/'hextof_config.yaml')b
# sp.save_energy_offset(filename=local_path/'hextof_config.yaml')
sp.save_workflow_params(filename=local_path/'hextof_config.yaml')

# Correct delay axis

In [ ]:
config_dict={
    "core": {
        "paths": {
            "data_raw_dir": "/asap3/flash/gpfs/pg2/2023/data/11019101/raw/hdf/offline/fl1user3", 
            "data_parquet_dir": "/home/agustsss/temp/sed_parquet/"
        },
        "binning":{"num_cores": 10}}}
sp = SedProcessor(
    runs=[44824,44825,44826,44827], 
    config=config_dict, 
    folder_config={},
    user_config=config_file, 
    collect_metadata=False
)

In [ ]:
sp.add_jitter()
sp.align_dld_sectors()
sp.append_energy_axis()
sp.add_energy_offset()

In [ ]:
axes = ['energy','delayStage']
bins = [100,150]
delay_start,delay_stop=1462.00,1464.85
ranges = [[-5,2], [delay_start,delay_stop]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)


In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,4), layout='constrained')
res.plot(robust=True, ax=ax[0])
bg = res.isel(delayStage=slice(0,10)).mean('delayStage')
(res-bg).plot(robust=True, ax=ax[1])

In [ ]:
sp._config["dataframe"]["delay_column"]

In [ ]:
sp.add_delay_offset(constant=-1463.7, flip_time_axis=True, columns=['bam'], signs=[0.001], preserve_mean=True)

In [ ]:
sp.dataframe # This has generated too many layers, there is room for improvement!

In [ ]:
axes = ['energy','delayStage']
bins = [100,150]
delay_start,delay_stop=1462.00,1464.85
ranges = [[-5,2], [-1.15, 1.7]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,5))
res.plot(robust=True, ax=ax[0])
bg = res.isel(delayStage=slice(0,10)).mean('delayStage')
(res-bg).plot(robust=True, ax=ax[1])

In [ ]:
sp.save_delay_offsets(filename=local_path/'hextof_config.yaml')

# load and process from config

In [ ]:
config_dict={"core": {"paths": {
    "data_raw_dir": "/asap3/flash/gpfs/pg2/2023/data/11019101/raw/hdf/offline/fl1user3", 
    "data_parquet_dir": "/home/agustsss/temp/sed_parquet/"
}}}
# config = sed.core.config.parse_config(config=config_dict, folder_config=local_path/'energy_calibration.yaml', user_config={}, system_config={})
sp = SedProcessor(runs=[44824,44825,44826,44827], config=config_dict, folder_config={}, user_config=config_file, collect_metadata=False)

In [ ]:
sp.add_jitter()
sp.align_dld_sectors()
sp.append_tof_ns_axis()
sp.append_energy_axis()
sp.add_energy_offset()
sp.add_delay_offset()

In [ ]:
axes = ['energy','delayStage']
bins = [100,150]
delay_start,delay_stop=1462.00,1464.85
ranges = [[-5,2], [-1.15, 1.7]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,5))
res.plot(robust=True, ax=ax[0])
bg = res.isel(delayStage=slice(0,10)).mean('delayStage')
(res-bg).plot(robust=True, ax=ax[1])

In [ ]:
sp.dataframe